In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar

## format table 2 (avg vol table)

In [39]:
table_2 = pd.read_csv("phase1_training/20min_avg_volume_training_phase1.csv")

In [40]:
table_2.head()

,tollgate_id,time_window,direction,volume
0,1,"[2016-09-19 00:00:00,2016-09-19 00:20:00)",1,140
1,1,"[2016-09-19 00:00:00,2016-09-19 00:20:00)",0,13
2,3,"[2016-09-19 00:00:00,2016-09-19 00:20:00)",1,181
3,3,"[2016-09-19 00:00:00,2016-09-19 00:20:00)",0,17
4,2,"[2016-09-19 00:00:00,2016-09-19 00:20:00)",0,2


In [41]:
start_date = pd.DataFrame(columns=["start_date"])
start_time = pd.DataFrame(columns=["start_time"])
end_date = pd.DataFrame(columns=["end_date"])
end_time = pd.DataFrame(columns=["end_time"])

In [42]:
for i in table_2.time_window:
    time_window_items = [j for i in [dt.split(" ") for dt in i[1:-1].split(",")] for j in i]
    start_date = start_date.append({'start_date': time_window_items[0]}, ignore_index=True)
    start_time = start_time.append({'start_time': time_window_items[1]}, ignore_index=True)
    end_date = end_date.append({'end_date': time_window_items[2]}, ignore_index=True)
    end_time = end_time.append({'end_time': time_window_items[3]}, ignore_index=True)

In [43]:
table_2["start_date"] = start_date
table_2["start_time"] = start_time
table_2["end_date"] = end_date
table_2["end_time"] = end_time

column_names = ["start_date", "start_time", "end_date", "end_time", "tollgate_id", "direction", "volume"]
table_2 = table_2.reindex(columns=column_names)
table_2.head()

,start_date,start_time,end_date,end_time,tollgate_id,direction,volume
0,2016-09-19,00:00:00,2016-09-19,00:20:00,1,1,140
1,2016-09-19,00:00:00,2016-09-19,00:20:00,1,0,13
2,2016-09-19,00:00:00,2016-09-19,00:20:00,3,1,181
3,2016-09-19,00:00:00,2016-09-19,00:20:00,3,0,17
4,2016-09-19,00:00:00,2016-09-19,00:20:00,2,0,2


In [44]:
table_2.count()

start_date     10063
start_time     10063
end_date       10063
end_time       10063
tollgate_id    10063
direction      10063
volume         10063
dtype: int64

## format table 7 (weather data)

In [45]:
table_7 = pd.read_csv("weather/weather_July_01_Oct_17_table7.csv")

In [46]:
table_7.head()

,date,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-07-01,0,1000.4,1005.3,225.0,2.1,26.4,94.0,0.0
1,2016-07-01,3,1000.5,1005.3,187.0,2.7,29.0,76.0,0.0
2,2016-07-01,6,998.9,1003.7,212.0,2.9,31.7,67.0,0.0
3,2016-07-01,9,998.7,1003.5,244.0,2.7,31.6,59.0,0.0
4,2016-07-01,12,999.7,1004.5,222.0,1.3,29.9,68.0,0.0


In [47]:
weather = pd.DataFrame()

In [48]:
new_time = pd.DataFrame(columns=["start_time"])

In [49]:
count = 0
for index, row in table_7.iterrows():
    record = row
    for delta in range(0,180,20):
        var = str(row["hour"])+"00"
        base_time = pd.to_datetime(var, format='%H%M%S')
        new_time = (base_time+dt.timedelta(minutes=delta)).time()
        temp = pd.Series([str(new_time)], index=["time"])
        new_record = pd.DataFrame([record.append(temp)]) 
        weather = pd.concat([new_record, weather], ignore_index=True)

In [50]:
weather = weather.drop(columns="hour")
weather.head()

,date,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,time
0,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,23:40:00
1,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,23:20:00
2,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,23:00:00
3,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,22:40:00
4,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,22:20:00


## combine table 2 and table 7

In [52]:
master_table = pd.DataFrame()

In [53]:
for v_idx, v_r in table_2.iterrows():
    for w_idx, w_r in weather.iterrows():
        if v_r["start_date"] == w_r["date"] and v_r["start_time"] == w_r["time"]:
            v_r = v_r.append(w_r)
            new_record = pd.DataFrame([v_r]) 
            master_table = pd.concat([new_record, master_table], ignore_index=True)

In [54]:
master_table.head()

,start_date,start_time,end_date,end_time,tollgate_id,direction,volume,date,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,time
0,2016-10-17,23:40:00,2016-10-18,00:00:00,2,0,18,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,23:40:00
1,2016-10-17,23:40:00,2016-10-18,00:00:00,3,0,15,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,23:40:00
2,2016-10-17,23:40:00,2016-10-18,00:00:00,3,1,22,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,23:40:00
3,2016-10-17,23:40:00,2016-10-18,00:00:00,1,0,2,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,23:40:00
4,2016-10-17,23:40:00,2016-10-18,00:00:00,1,1,10,2016-10-17,1013.9,1018.9,322.0,2.5,19.3,90.0,0.0,23:40:00


In [56]:
master_table.columns

Index(['start_date', 'start_time', 'end_date', 'end_time', 'tollgate_id',
       'direction', 'volume', 'date', 'pressure', 'sea_pressure',
       'wind_direction', 'wind_speed', 'temperature', 'rel_humidity',
       'precipitation', 'time'],
      dtype='object')

In [57]:
column_names = ['start_date', 'start_time', 'end_date', 'end_time', 'tollgate_id', 'direction', 'precipitation', 'pressure', 'rel_humidity', 
 'sea_pressure', 'temperature', 'wind_direction', 'wind_speed', 'volume']
master_table = master_table.reindex(columns=column_names)
master_table.head()

,start_date,start_time,end_date,end_time,tollgate_id,direction,precipitation,pressure,rel_humidity,sea_pressure,temperature,wind_direction,wind_speed,volume
0,2016-10-17,23:40:00,2016-10-18,00:00:00,2,0,0.0,1013.9,90.0,1018.9,19.3,322.0,2.5,18
1,2016-10-17,23:40:00,2016-10-18,00:00:00,3,0,0.0,1013.9,90.0,1018.9,19.3,322.0,2.5,15
2,2016-10-17,23:40:00,2016-10-18,00:00:00,3,1,0.0,1013.9,90.0,1018.9,19.3,322.0,2.5,22
3,2016-10-17,23:40:00,2016-10-18,00:00:00,1,0,0.0,1013.9,90.0,1018.9,19.3,322.0,2.5,2
4,2016-10-17,23:40:00,2016-10-18,00:00:00,1,1,0.0,1013.9,90.0,1018.9,19.3,322.0,2.5,10


In [58]:
master_table.to_csv(path_or_buf="task_2_master_table_training.csv")

In [59]:
data = pd.read_csv("task_2_master_table_training.csv")

In [60]:
data.count()

Unnamed: 0        9630
start_date        9630
start_time        9630
end_date          9630
end_time          9630
tollgate_id       9630
direction         9630
precipitation     9630
pressure          9630
rel_humidity      9630
sea_pressure      9630
temperature       9630
wind_direction    9630
wind_speed        9630
volume            9630
dtype: int64

## formatting table 6

In [10]:
t6_data = pd.read_csv("phase1_training/new_volume_training_phase1_table6.csv")

In [11]:
t6_data.head()

,Unnamed: 0,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,330989,2016-09-19 23:18:06,1,1,1,1,0.0
1,330990,2016-09-19 22:31:52,3,1,1,1,0.0
2,330991,2016-09-19 22:21:18,1,1,1,1,0.0
3,330992,2016-09-19 21:21:37,1,1,2,1,0.0
4,330993,2016-09-19 23:22:09,1,1,1,1,0.0


In [14]:
t6_data.count()

Unnamed: 0       543699
time             543699
tollgate_id      543699
direction        543699
vehicle_model    543699
has_etc          543699
vehicle_type     543699
dtype: int64

In [12]:
date = pd.DataFrame(columns=["date"])
time = pd.DataFrame(columns=["time"])

In [ ]:
for i in t6_data.time:
    time_items = [x for x in i.split(" ")]
    start_date = start_date.append({'date': time_items[0]}, ignore_index=True)
    start_time = start_time.append({'time': time_items[1]}, ignore_index=True)

In [66]:
new_tab = t6_data.drop(['time'], axis=1)
new_tab["date"] = start_date
new_tab["time"] = start_time

In [67]:
new_tab.head()

,Unnamed: 0,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,date,time
0,330989,1,1,1,1,0.0,2016-09-19,23:18:06
1,330990,3,1,1,1,0.0,2016-09-19,22:31:52
2,330991,1,1,1,1,0.0,2016-09-19,22:21:18
3,330992,1,1,2,1,0.0,2016-09-19,21:21:37
4,330993,1,1,1,1,0.0,2016-09-19,23:22:09


## test data formatting

## table 2 (test) avg vol

In [16]:
test_2 = pd.read_csv("phase1_test/20min_avg_volume_test_phase1.csv")

In [17]:
test_2.head()

,tollgate_id,time_window,direction,volume
0,1,"[2016-10-18 06:00:00,2016-10-18 06:20:00)",1,NaN
1,1,"[2016-10-18 06:00:00,2016-10-18 06:20:00)",0,NaN
2,3,"[2016-10-18 06:00:00,2016-10-18 06:20:00)",1,NaN
3,3,"[2016-10-18 06:00:00,2016-10-18 06:20:00)",0,NaN
4,2,"[2016-10-18 06:00:00,2016-10-18 06:20:00)",0,NaN


In [18]:
test_2.count()

tollgate_id    420
time_window    420
direction      420
volume           0
dtype: int64

In [19]:
start_date = pd.DataFrame(columns=["start_date"])
start_time = pd.DataFrame(columns=["start_time"])
end_date = pd.DataFrame(columns=["end_date"])
end_time = pd.DataFrame(columns=["end_time"])

In [20]:
for i in test_2.time_window:
    time_window_items = [j for i in [dt.split(" ") for dt in i[1:-1].split(",")] for j in i]
    start_date = start_date.append({'start_date': time_window_items[0]}, ignore_index=True)
    start_time = start_time.append({'start_time': time_window_items[1]}, ignore_index=True)
    end_date = end_date.append({'end_date': time_window_items[2]}, ignore_index=True)
    end_time = end_time.append({'end_time': time_window_items[3]}, ignore_index=True)

In [21]:
test_2["start_date"] = start_date
test_2["start_time"] = start_time
test_2["end_date"] = end_date
test_2["end_time"] = end_time

column_names = ["start_date", "start_time", "end_date", "end_time", "tollgate_id", "direction", "volume"]
test_2 = test_2.reindex(columns=column_names)
test_2.head()

,start_date,start_time,end_date,end_time,tollgate_id,direction,volume
0,2016-10-18,06:00:00,2016-10-18,06:20:00,1,1,NaN
1,2016-10-18,06:00:00,2016-10-18,06:20:00,1,0,NaN
2,2016-10-18,06:00:00,2016-10-18,06:20:00,3,1,NaN
3,2016-10-18,06:00:00,2016-10-18,06:20:00,3,0,NaN
4,2016-10-18,06:00:00,2016-10-18,06:20:00,2,0,NaN


In [22]:
test_2.count()

start_date     420
start_time     420
end_date       420
end_time       420
tollgate_id    420
direction      420
volume           0
dtype: int64

## formatting table 7 (test data)

In [23]:
test_7 = pd.read_csv("weather/weather_Oct_18_Oct_24_table7.csv")

In [24]:
test_7.head()

,date,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-10-18,0,1015.2,1020.3,62.0,2.1,20.5,87.0,0.0
1,2016-10-18,3,1014.5,1019.5,37.0,2.5,23.3,73.0,0.0
2,2016-10-18,6,1012.5,1017.5,128.0,1.6,23.4,73.0,0.0
3,2016-10-18,9,1013.0,1018.0,205.0,0.6,20.9,94.0,1.8
4,2016-10-18,12,1013.8,1018.8,106.0,1.5,20.5,94.0,0.5


In [25]:
t_weather = pd.DataFrame()

In [26]:
t_new_time = pd.DataFrame(columns=["start_time"])

In [27]:
count = 0
for index, row in test_7.iterrows():
    record = row
    for delta in range(0,180,20):
        var = str(row["hour"])+"00"
        base_time = pd.to_datetime(var, format='%H%M%S')
        t_new_time = (base_time+dt.timedelta(minutes=delta)).time()
        temp = pd.Series([str(t_new_time)], index=["time"])
        new_record = pd.DataFrame([record.append(temp)]) 
        t_weather = pd.concat([new_record, t_weather], ignore_index=True)

In [28]:
t_weather = t_weather.drop(columns="hour")
t_weather.head()

,date,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,time
0,2016-10-24,1008.1,1013.1,323.0,2.5,20.8,93.0,0.0,23:40:00
1,2016-10-24,1008.1,1013.1,323.0,2.5,20.8,93.0,0.0,23:20:00
2,2016-10-24,1008.1,1013.1,323.0,2.5,20.8,93.0,0.0,23:00:00
3,2016-10-24,1008.1,1013.1,323.0,2.5,20.8,93.0,0.0,22:40:00
4,2016-10-24,1008.1,1013.1,323.0,2.5,20.8,93.0,0.0,22:20:00


## combine table 2 and table 7 (test data)

In [29]:
test_master_table = pd.DataFrame()

In [30]:
for v_idx, v_r in test_2.iterrows():
    for w_idx, w_r in t_weather.iterrows():
        if v_r["start_date"] == w_r["date"] and v_r["start_time"] == w_r["time"]:
            v_r = v_r.append(w_r)
            new_record = pd.DataFrame([v_r]) 
            test_master_table = pd.concat([new_record, test_master_table], ignore_index=True)

In [31]:
test_master_table.head()

,start_date,start_time,end_date,end_time,tollgate_id,direction,volume,date,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,time
0,2016-10-24,16:40:00,2016-10-24,17:00:00,2,0,NaN,2016-10-24,1008.0,1013.0,210.0,1.4,21.0,91.0,0.0,16:40:00
1,2016-10-24,16:40:00,2016-10-24,17:00:00,3,0,NaN,2016-10-24,1008.0,1013.0,210.0,1.4,21.0,91.0,0.0,16:40:00
2,2016-10-24,16:40:00,2016-10-24,17:00:00,3,1,NaN,2016-10-24,1008.0,1013.0,210.0,1.4,21.0,91.0,0.0,16:40:00
3,2016-10-24,16:40:00,2016-10-24,17:00:00,1,0,NaN,2016-10-24,1008.0,1013.0,210.0,1.4,21.0,91.0,0.0,16:40:00
4,2016-10-24,16:40:00,2016-10-24,17:00:00,1,1,NaN,2016-10-24,1008.0,1013.0,210.0,1.4,21.0,91.0,0.0,16:40:00


In [32]:
test_master_table.columns

Index(['start_date', 'start_time', 'end_date', 'end_time', 'tollgate_id',
       'direction', 'volume', 'date', 'pressure', 'sea_pressure',
       'wind_direction', 'wind_speed', 'temperature', 'rel_humidity',
       'precipitation', 'time'],
      dtype='object')

In [33]:
column_names = ['start_date', 'start_time', 'end_date', 'end_time', 'tollgate_id', 'direction', 'precipitation', 'pressure', 'rel_humidity', 
 'sea_pressure', 'temperature', 'wind_direction', 'wind_speed', 'volume']
test_master_table = test_master_table.reindex(columns=column_names)
test_master_table.head()

,start_date,start_time,end_date,end_time,tollgate_id,direction,precipitation,pressure,rel_humidity,sea_pressure,temperature,wind_direction,wind_speed,volume
0,2016-10-24,16:40:00,2016-10-24,17:00:00,2,0,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN
1,2016-10-24,16:40:00,2016-10-24,17:00:00,3,0,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN
2,2016-10-24,16:40:00,2016-10-24,17:00:00,3,1,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN
3,2016-10-24,16:40:00,2016-10-24,17:00:00,1,0,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN
4,2016-10-24,16:40:00,2016-10-24,17:00:00,1,1,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN


In [34]:
test_master_table.to_csv(path_or_buf="task_2_test_master_table_training.csv")

In [35]:
t_data = pd.read_csv("task_2_test_master_table_training.csv")

In [36]:
t_data.head()

,Unnamed: 0,start_date,start_time,end_date,end_time,tollgate_id,direction,precipitation,pressure,rel_humidity,sea_pressure,temperature,wind_direction,wind_speed,volume
0,0,2016-10-24,16:40:00,2016-10-24,17:00:00,2,0,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN
1,1,2016-10-24,16:40:00,2016-10-24,17:00:00,3,0,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN
2,2,2016-10-24,16:40:00,2016-10-24,17:00:00,3,1,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN
3,3,2016-10-24,16:40:00,2016-10-24,17:00:00,1,0,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN
4,4,2016-10-24,16:40:00,2016-10-24,17:00:00,1,1,0.0,1008.0,91.0,1013.0,21.0,210.0,1.4,NaN


In [37]:
t_data.count()

Unnamed: 0        420
start_date        420
start_time        420
end_date          420
end_time          420
tollgate_id       420
direction         420
precipitation     420
pressure          420
rel_humidity      420
sea_pressure      420
temperature       420
wind_direction    420
wind_speed        420
volume              0
dtype: int64

## adding day of week to train master table

In [2]:
t_data = pd.read_csv("data/preprocessing_data/task_2_master_table_training.csv")

In [31]:
calendar.setfirstweekday(calendar.MONDAY)

In [33]:
weekday = list()

for idx, row in t_data.iterrows():
    date_part = [int(i) for i in row["start_date"].split("-")]
    weekday.append(calendar.weekday(date_part[0], date_part[1], date_part[2]))


In [34]:
t_data["weekday"] = weekday

In [35]:
t_data.head()

,Unnamed: 0,start_date,start_time,end_date,end_time,tollgate_id,direction,precipitation,pressure,rel_humidity,sea_pressure,temperature,wind_direction,wind_speed,volume,weekday
0,0,2016-10-17,23:40:00,2016-10-18,00:00:00,2,0,0.0,1013.9,90,1018.9,19.3,322,2.5,18,0
1,1,2016-10-17,23:40:00,2016-10-18,00:00:00,3,0,0.0,1013.9,90,1018.9,19.3,322,2.5,15,0
2,2,2016-10-17,23:40:00,2016-10-18,00:00:00,3,1,0.0,1013.9,90,1018.9,19.3,322,2.5,22,0
3,3,2016-10-17,23:40:00,2016-10-18,00:00:00,1,0,0.0,1013.9,90,1018.9,19.3,322,2.5,2,0
4,4,2016-10-17,23:40:00,2016-10-18,00:00:00,1,1,0.0,1013.9,90,1018.9,19.3,322,2.5,10,0


In [36]:
t_data.to_csv(path_or_buf="data/preprocessing_data/task_2_master_table_training.csv")

## adding day of week to test master table

In [16]:
data = pd.read_csv("data/preprocessing_data/task_2_master_table_test.csv")

In [17]:
data.head()

,Unnamed: 0,Unnamed: 0.1,start_date,start_time,end_date,end_time,tollgate_id,direction,precipitation,pressure,rel_humidity,sea_pressure,temperature,wind_direction,wind_speed,volume
0,0,0,2016-10-24,16:40:00,2016-10-24,17:00:00,2,0,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN
1,1,1,2016-10-24,16:40:00,2016-10-24,17:00:00,3,0,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN
2,2,2,2016-10-24,16:40:00,2016-10-24,17:00:00,3,1,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN
3,3,3,2016-10-24,16:40:00,2016-10-24,17:00:00,1,0,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN
4,4,4,2016-10-24,16:40:00,2016-10-24,17:00:00,1,1,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN


In [18]:
calendar.setfirstweekday(calendar.MONDAY)

In [19]:
weekday = list()

for idx, row in data.iterrows():
    date_part = [int(i) for i in row["start_date"].split("-")]
    weekday.append(calendar.weekday(date_part[0], date_part[1], date_part[2]))


In [20]:
data["weekday"] = weekday

In [21]:
data.head()

,Unnamed: 0,Unnamed: 0.1,start_date,start_time,end_date,end_time,tollgate_id,direction,precipitation,pressure,rel_humidity,sea_pressure,temperature,wind_direction,wind_speed,volume,weekday
0,0,0,2016-10-24,16:40:00,2016-10-24,17:00:00,2,0,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN,0
1,1,1,2016-10-24,16:40:00,2016-10-24,17:00:00,3,0,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN,0
2,2,2,2016-10-24,16:40:00,2016-10-24,17:00:00,3,1,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN,0
3,3,3,2016-10-24,16:40:00,2016-10-24,17:00:00,1,0,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN,0
4,4,4,2016-10-24,16:40:00,2016-10-24,17:00:00,1,1,0.0,1008.0,91,1013.0,21.0,210,1.4,NaN,0


In [22]:
data.to_csv(path_or_buf="data/preprocessing_data/task_2_master_table_test.csv")

In [23]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'start_date', 'start_time', 'end_date',
       'end_time', 'tollgate_id', 'direction', 'precipitation', 'pressure',
       'rel_humidity', 'sea_pressure', 'temperature', 'wind_direction',
       'wind_speed', 'volume', 'weekday'],
      dtype='object')

## Splitting time and date into separate columns of train master table

In [41]:
data = pd.read_csv("data/preprocessing_data/task_2_master_table_training.csv")

In [42]:
data = data.drop(["Unnamed: 0"],axis=1)

In [43]:
data.columns

Index(['Unnamed: 0.1', 'start_date', 'start_time', 'end_date', 'end_time',
       'tollgate_id', 'direction', 'precipitation', 'pressure', 'rel_humidity',
       'sea_pressure', 'temperature', 'wind_direction', 'wind_speed', 'volume',
       'weekday'],
      dtype='object')

In [48]:
features = data

In [49]:
def split_date_to_year_month_date(features, which_date):
    date_cols = [ x for x in [i.split("-") for i in features[which_date] ]]
    date_year = [ float(i[0]) for i in date_cols ] 
    date_month = [ float(i[1]) for i in date_cols ] 
    date_days = [ float(i[2]) for i in date_cols ]
    return date_year, date_month, date_days

In [50]:
date_year, date_month, date_days = split_date_to_year_month_date(features,'start_date')

In [51]:
features["start_date_year"] = date_year
features["start_date_month"] = date_month
features["start_date_day"] = date_days

In [52]:
date_year, date_month, date_days = split_date_to_year_month_date(features,'end_date')

features["end_date_year"] = date_year
features["end_date_month"] = date_month
features["end_date_day"] = date_days

In [53]:
features.head()

,Unnamed: 0.1,start_date,start_time,end_date,end_time,tollgate_id,direction,precipitation,pressure,rel_humidity,...,wind_direction,wind_speed,volume,weekday,start_date_year,start_date_month,start_date_day,end_date_year,end_date_month,end_date_day
0,0,2016-10-17,23:40:00,2016-10-18,00:00:00,2,0,0.0,1013.9,90,...,322,2.5,18,0,2016.0,10.0,17.0,2016.0,10.0,18.0
1,1,2016-10-17,23:40:00,2016-10-18,00:00:00,3,0,0.0,1013.9,90,...,322,2.5,15,0,2016.0,10.0,17.0,2016.0,10.0,18.0
2,2,2016-10-17,23:40:00,2016-10-18,00:00:00,3,1,0.0,1013.9,90,...,322,2.5,22,0,2016.0,10.0,17.0,2016.0,10.0,18.0
3,3,2016-10-17,23:40:00,2016-10-18,00:00:00,1,0,0.0,1013.9,90,...,322,2.5,2,0,2016.0,10.0,17.0,2016.0,10.0,18.0
4,4,2016-10-17,23:40:00,2016-10-18,00:00:00,1,1,0.0,1013.9,90,...,322,2.5,10,0,2016.0,10.0,17.0,2016.0,10.0,18.0


In [54]:
features = features.drop(["start_date","end_date"],axis=1)

In [55]:
end_time_cols = [ x[:-1] for x in [i.split(":") for i in features.end_time ]]
end_time_hour = [ float(i[0]) for i in end_time_cols ] 
end_time_min = [ float(i[1]) for i in end_time_cols ] 
features["end_time_hour"] = end_time_hour
features["end_time_min"] = end_time_min

In [56]:
start_time_cols = [ x[:-1] for x in [i.split(":") for i in features.start_time ]]
start_time_hour = [ float(i[0]) for i in start_time_cols ] 
start_time_min = [ float(i[1]) for i in start_time_cols ] 
features["start_time_hour"] = start_time_hour
features["start_time_min"] = start_time_min

In [57]:
features = features.drop(["end_time","start_time"],axis=1)

In [58]:
features.columns

Index(['Unnamed: 0.1', 'tollgate_id', 'direction', 'precipitation', 'pressure',
       'rel_humidity', 'sea_pressure', 'temperature', 'wind_direction',
       'wind_speed', 'volume', 'weekday', 'start_date_year',
       'start_date_month', 'start_date_day', 'end_date_year', 'end_date_month',
       'end_date_day', 'end_time_hour', 'end_time_min', 'start_time_hour',
       'start_time_min'],
      dtype='object')

In [59]:
features.to_csv(path_or_buf="data/preprocessing_data/task_2_master_table_training.csv")

## Splitting time and date into separate columns of test master table

In [60]:
data = pd.read_csv("data/preprocessing_data/task_2_master_table_test.csv")

In [61]:
data = data.drop(["Unnamed: 0"],axis=1)

In [62]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0.1.1', 'start_date', 'start_time',
       'end_date', 'end_time', 'tollgate_id', 'direction', 'precipitation',
       'pressure', 'rel_humidity', 'sea_pressure', 'temperature',
       'wind_direction', 'wind_speed', 'volume', 'weekday'],
      dtype='object')

In [63]:
features = data

In [64]:
date_year, date_month, date_days = split_date_to_year_month_date(features,'start_date')

In [65]:
features["start_date_year"] = date_year
features["start_date_month"] = date_month
features["start_date_day"] = date_days

In [66]:
date_year, date_month, date_days = split_date_to_year_month_date(features,'end_date')

features["end_date_year"] = date_year
features["end_date_month"] = date_month
features["end_date_day"] = date_days

In [67]:
features = features.drop(["start_date","end_date"],axis=1)

In [68]:
end_time_cols = [ x[:-1] for x in [i.split(":") for i in features.end_time ]]
end_time_hour = [ float(i[0]) for i in end_time_cols ] 
end_time_min = [ float(i[1]) for i in end_time_cols ] 
features["end_time_hour"] = end_time_hour
features["end_time_min"] = end_time_min

In [69]:
start_time_cols = [ x[:-1] for x in [i.split(":") for i in features.start_time ]]
start_time_hour = [ float(i[0]) for i in start_time_cols ] 
start_time_min = [ float(i[1]) for i in start_time_cols ] 
features["start_time_hour"] = start_time_hour
features["start_time_min"] = start_time_min

In [70]:
features = features.drop(["end_time","start_time"],axis=1)

In [71]:
features.columns

Index(['Unnamed: 0.1', 'Unnamed: 0.1.1', 'tollgate_id', 'direction',
       'precipitation', 'pressure', 'rel_humidity', 'sea_pressure',
       'temperature', 'wind_direction', 'wind_speed', 'volume', 'weekday',
       'start_date_year', 'start_date_month', 'start_date_day',
       'end_date_year', 'end_date_month', 'end_date_day', 'end_time_hour',
       'end_time_min', 'start_time_hour', 'start_time_min'],
      dtype='object')

In [72]:
features.to_csv(path_or_buf="data/preprocessing_data/task_2_master_table_test.csv")